# **Sistem Rekomendasi Pencarian Magang Merdeka Content Based Filtering**

## **Data Scraping**

In [1]:
import requests
import pandas as pd
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time

In [12]:
url_opportunities = "https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/opportunities"
url_detail = "https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position/{}"
limit = 100
offset = 0

all_data = pd.DataFrame()

while True:
    params = {
        "offset": offset,
        "limit": limit,
        "location_key": "",
        "mitra_key": "",
        "keyword": "",
        "sector_id": "",
        "sort_by": "",
        "order": "desc"
    }

    response = requests.get(url_opportunities, params=params)

    offset += limit
    data = response.json()["data"]
    if len(data) == 0:
        break

    # Mengupulkan detail data setiap id entry
    for entry_id in data:
        detail_url = url_detail.format(entry_id["id"])

        detail_response = requests.get(detail_url)

        detail_data = pd.json_normalize(detail_response.json())

        # Menghilangkan prefix "data."
        detail_data.columns = [col.replace("data.", "").replace("benefits.", "") for col in detail_data.columns]

        all_data = pd.concat([all_data, detail_data], ignore_index=True)

    print(f"Data collected: {len(all_data)}")

print(f"Total {len(all_data)} data collected")

ValueError: Array conditional must be same shape as self

In [9]:
all_data

,id,opportunity_type,name,activity_type,location,months_duration,start_registration,end_registration,mitra_id,mitra_name,...,credits_count,activity_type,location,location_kotakab_code,mitra_id,certified,skills,available_to_apply,is_quota_full,activity_active_id
0,190c81c5-048a-11ef-9e17-0a54edb93563,MSIB,Layanan Publik,WFO,Kota Yogyakarta,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,4afed716-68b7-48d4-bddb-6fa73a3433bc,Dinas Lingkungan Hidup Dan Kehutanan Daerah Is...,...,20,OFFLINE,Kota Yogyakarta,046000,4afed716-68b7-48d4-bddb-6fa73a3433bc,True,[{'id': '57ddce5c-048b-11ef-991a-ae222c5b6357'...,True,False,0
1,1b162247-048d-11ef-bb03-ceeddaa1b367,MSIB,Pengelola Arsip,WFO,Kota Yogyakarta,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,4afed716-68b7-48d4-bddb-6fa73a3433bc,Dinas Lingkungan Hidup Dan Kehutanan Daerah Is...,...,20,OFFLINE,Kota Yogyakarta,046000,4afed716-68b7-48d4-bddb-6fa73a3433bc,True,[{'id': '6c0e2f08-048e-11ef-bb03-ceeddaa1b367'...,True,False,0
2,f2b8c752-04a0-11ef-8394-36f764739585,MSIB,Library Quality Analysis Staff and Integrated ...,WFO,Kota Yogyakarta,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,4afed716-68b7-48d4-bddb-6fa73a3433bc,Dinas Lingkungan Hidup Dan Kehutanan Daerah Is...,...,20,OFFLINE,Kota Yogyakarta,046000,4afed716-68b7-48d4-bddb-6fa73a3433bc,True,[{'id': 'ee5cb0e2-04a5-11ef-8394-36f764739585'...,True,False,0
3,90665e71-0490-11ef-bb7f-6ac06bc5928b,MSIB,Staf Manajemen,WFO,Kota Yogyakarta,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,4afed716-68b7-48d4-bddb-6fa73a3433bc,Dinas Lingkungan Hidup Dan Kehutanan Daerah Is...,...,20,OFFLINE,Kota Yogyakarta,046000,4afed716-68b7-48d4-bddb-6fa73a3433bc,True,[{'id': '9075a8e1-0492-11ef-9b3e-022ae5f04a49'...,True,False,0
4,60636725-0494-11ef-9e17-0a54edb93563,MSIB,Full-stack Developer,WFO,Kota Yogyakarta,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,4afed716-68b7-48d4-bddb-6fa73a3433bc,Dinas Lingkungan Hidup Dan Kehutanan Daerah Is...,...,20,OFFLINE,Kota Yogyakarta,046000,4afed716-68b7-48d4-bddb-6fa73a3433bc,True,[{'id': '393df447-0495-11ef-bb03-ceeddaa1b367'...,True,False,0
5,ecde7bfd-046b-11ef-a051-e22fd7765666,MSIB,Pustakawan,WFO,Kota Yogyakarta,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,4afed716-68b7-48d4-bddb-6fa73a3433bc,Dinas Lingkungan Hidup Dan Kehutanan Daerah Is...,...,20,OFFLINE,Kota Yogyakarta,046000,4afed716-68b7-48d4-bddb-6fa73a3433bc,True,[{'id': '7409a14f-0483-11ef-bb03-ceeddaa1b367'...,True,False,0
6,fc9aeb30-fddf-11ee-8a41-f67d68bd6616,MSIB,Desain Grafis,WFO,Kota Makassar,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,d391d5b2-22ae-4ed2-be87-3139125b6c96,Lembaga Layanan Pendidikan Tinggi Wilayah IX,...,20,OFFLINE,Kota Makassar,196000,d391d5b2-22ae-4ed2-be87-3139125b6c96,True,[{'id': '85275128-fde0-11ee-8a41-f67d68bd6616'...,True,False,0
7,17fa594c-fb88-11ee-82ee-42fdda477ac9,MSIB,Backend Programmer,WFO,Kota Makassar,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,d391d5b2-22ae-4ed2-be87-3139125b6c96,Lembaga Layanan Pendidikan Tinggi Wilayah IX,...,20,OFFLINE,Kota Makassar,196000,d391d5b2-22ae-4ed2-be87-3139125b6c96,True,[{'id': '3189f4e2-fb8a-11ee-82ee-42fdda477ac9'...,True,False,0
8,05d2701e-f179-11ee-af0a-42786e9b30fb,MSIB,Frontend Programmer,WFO,Kota Makassar,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,d391d5b2-22ae-4ed2-be87-3139125b6c96,Lembaga Layanan Pendidikan Tinggi Wilayah IX,...,20,OFFLINE,Kota Makassar,196000,d391d5b2-22ae-4ed2-be87-3139125b6c96,True,[{'id': '40e9d3dd-f17d-11ee-bf75-32cfa9fc582e'...,True,False,0
9,9a6a0db3-fde0-11ee-865e-fae64678b8c1,MSIB,Editor Vidio,WFO,Kota Makassar,4,2024-04-16T00:00:00Z,2024-06-10T00:00:00Z,d391d5b2-22ae-4ed2-be87-3139125b6c96,Lembaga Layanan Pendidikan Tinggi Wilayah IX,...,20,OFFLINE,Kota Makassar,196000,d391d5b2-22ae-4ed2-be87-3139125b6c96,True,[{'id': '44c0efcb-fde1-11ee-8a41-f67d68bd6616'...,True,False,0


In [10]:
all_data.to_csv('../data/magang_opportunities3.csv', index=False)

## **Data PreProcessing**

### **Data Reviewing**

In [19]:
import pandas as pd

In [20]:
magang_opportunities = pd.read_csv('../data/magang_opportunities.csv')

magang_opportunities

,meta,id,name,description,mitra_id,start_period,months_duration,activity_type,location,fields_of_study,...,show_salary,mobilization,accommodation,is_applied,wishlist,is_external,external_platform_name,external_platform_logo_url,mitra_name,salary
0,NaN,b794e1a1-ecd6-449d-9ed9-0118a2e02626,Website Developer,<p><strong>Kualifikasi :</strong>&nbsp;</p><ul...,c7746ff1-87de-47bb-a43f-6d63f521e07c,2024-03-01T00:00:00+07:00,3,WFH,Kota Surabaya,[],...,False,False,False,False,False,False,NaN,NaN,CV. APPAREL BERKAH SELALU,NaN
1,NaN,7c8623ef-5f9b-470d-bc7a-73b1dd706c8b,Software Architect Intern,<p>To help architect team on demonstration of ...,af00bd20-3ac2-44a4-877f-af4307f9b599,2024-03-01T00:00:00+07:00,6,WFO,Kota Jakarta Selatan,['Teknik Informatika dan Ilmu Komputer'],...,False,False,False,False,False,False,NaN,NaN,PT. Idemia Technologies Indonesia,NaN
2,NaN,bd95c730-b157-4fc5-8791-9dc9ea5e0e32,Product Marketing Intern,<p>- Design and develop sharepoint page to sup...,af00bd20-3ac2-44a4-877f-af4307f9b599,2024-03-01T00:00:00+07:00,6,WFO,Kota Jakarta Selatan,['Teknik Informatika dan Ilmu Komputer'],...,False,False,False,False,False,False,NaN,NaN,PT. Idemia Technologies Indonesia,NaN
3,NaN,fc01895c-0d74-4874-a8f2-d1d46956d697,Asisten Finance,<ol><li>Pemahaman Sistem Keuangan Sekolah: And...,0bf5a4d1-9980-48ae-ad65-48e3b239f15c,2024-03-01T00:00:00+07:00,4,WFO,Kota Banda Aceh,[],...,False,False,False,False,False,False,NaN,NaN,SMAN 14 Banda Aceh,NaN
4,NaN,b6062e51-4764-474c-a2b7-617affd56959,Marketing Asuransi Mini Mikro,<p>WFH dari Seluruh Indonesia.</p><p><br></p><...,6bbc729e-c3ee-46a1-93be-2b9f3b5f478d,2024-03-01T00:00:00+07:00,4,WFH,Kota Banda Aceh,[],...,False,False,False,False,False,False,NaN,NaN,Natural Aceh,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,NaN,7a2f6ef1-844d-4c70-99bc-03be7d5aac88,Marketing Creative Intern,Responsibilities \r\n\r\nSupport Marketing and...,ee2084cf-f2b4-436a-b6a1-9e23d2e9827a,2024-02-06T07:28:52.149199+07:00,0,WFO,Tanah Abang,NaN,...,True,False,False,False,False,True,KitaLulus,https://i.imgur.com/1WEx0zY.png,The Jakarta Post,0.0
678,NaN,404ccd37-0c1e-43ab-bdfd-7b9a6fee551c,Graphic Design (Intern),Kami Idea merupakan sebuah usaha atau bisnis y...,0e19e313-3c8c-46ad-bb18-07e5a4fc2ef4,2024-02-06T07:28:52.149199+07:00,0,WFO,Mampang Prapatan,NaN,...,True,False,False,False,False,True,KitaLulus,https://i.imgur.com/1WEx0zY.png,Kami Idea,0.0
679,NaN,5a704786-6865-40c9-9086-a2bd85dc5098,Influencer Management Intern,- Last semester student from reputable univers...,32ac867c-45a6-4752-b7a9-23cbad490916,2024-02-06T07:28:52.149199+07:00,0,WFO,Tanjung Priok,NaN,...,True,False,False,False,False,True,KitaLulus,https://i.imgur.com/1WEx0zY.png,Deca Group,0.0
680,NaN,780902fd-e51a-463c-a012-e7a0941da47e,New Business Development Internship (Jakarta U...,- Final year student or fresh graduate \r\n- P...,e11ceed4-4513-4073-8674-eb7950a05c2b,2024-02-06T07:28:52.149199+07:00,0,WFO,Kelapa Gading,NaN,...,True,False,False,False,False,True,KitaLulus,https://i.imgur.com/1WEx0zY.png,Sociolla,0.0


In [21]:
data = pd.DataFrame({
  'id': list(magang_opportunities['id'].values),
  'name': list(magang_opportunities['name'].values),
  'description': list(magang_opportunities['description'].values),
  'skills': list(magang_opportunities['skills'].values)
})

data

,id,name,description,skills
0,b794e1a1-ecd6-449d-9ed9-0118a2e02626,Website Developer,<p><strong>Kualifikasi :</strong>&nbsp;</p><ul...,"['Komunikasi', 'Kreatif', 'Pemahaman Industri'..."
1,7c8623ef-5f9b-470d-bc7a-73b1dd706c8b,Software Architect Intern,<p>To help architect team on demonstration of ...,"['Komunikasi', 'Kreatif', 'Desain Grafis', 'Co..."
2,bd95c730-b157-4fc5-8791-9dc9ea5e0e32,Product Marketing Intern,<p>- Design and develop sharepoint page to sup...,"['Kreatif', 'Analisis Data', 'Pemahaman Lingku..."
3,fc01895c-0d74-4874-a8f2-d1d46956d697,Asisten Finance,<ol><li>Pemahaman Sistem Keuangan Sekolah: And...,"['Administrasi', 'Operasional', 'Komunikasi', ..."
4,b6062e51-4764-474c-a2b7-617affd56959,Marketing Asuransi Mini Mikro,<p>WFH dari Seluruh Indonesia.</p><p><br></p><...,"['Operasional', 'Kreatif', 'Administrasi', 'Ko..."
...,...,...,...,...
677,7a2f6ef1-844d-4c70-99bc-03be7d5aac88,Marketing Creative Intern,Responsibilities \r\n\r\nSupport Marketing and...,NaN
678,404ccd37-0c1e-43ab-bdfd-7b9a6fee551c,Graphic Design (Intern),Kami Idea merupakan sebuah usaha atau bisnis y...,NaN
679,5a704786-6865-40c9-9086-a2bd85dc5098,Influencer Management Intern,- Last semester student from reputable univers...,NaN
680,780902fd-e51a-463c-a012-e7a0941da47e,New Business Development Internship (Jakarta U...,- Final year student or fresh graduate \r\n- P...,NaN


### **Combine Text**

In [22]:
import ast

In [23]:
# Convert the "skills" column from string to list for non-null values
data['skills'] = data['skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

# Join the skills into a single string
data['skills'] = data['skills'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Combine "name," "description," and "skills" into a new column
data['combined_text'] = data['name'] + ' ' + data['description'] + ' ' + data['skills'].astype(str)

data['combined_text']

0      Website Developer <p><strong>Kualifikasi :</st...
1      Software Architect Intern <p>To help architect...
2      Product Marketing Intern <p>- Design and devel...
3      Asisten Finance <ol><li>Pemahaman Sistem Keuan...
4      Marketing Asuransi Mini Mikro  <p>WFH dari Sel...
                             ...                        
677    Marketing Creative Intern Responsibilities \r\...
678    Graphic Design (Intern) Kami Idea merupakan se...
679    Influencer Management Intern - Last semester s...
680    New Business Development Internship (Jakarta U...
681    Finance & Accounting Intern Kualifikasi :\r\n-...
Name: combined_text, Length: 682, dtype: object

### **Case Folding**

In [24]:
data['result_case_folding_data'] = data['combined_text'].apply(lambda x: x.casefold())
data[['id', 'combined_text','result_case_folding_data']]

,id,combined_text,result_case_folding_data
0,b794e1a1-ecd6-449d-9ed9-0118a2e02626,Website Developer <p><strong>Kualifikasi :</st...,website developer <p><strong>kualifikasi :</st...
1,7c8623ef-5f9b-470d-bc7a-73b1dd706c8b,Software Architect Intern <p>To help architect...,software architect intern <p>to help architect...
2,bd95c730-b157-4fc5-8791-9dc9ea5e0e32,Product Marketing Intern <p>- Design and devel...,product marketing intern <p>- design and devel...
3,fc01895c-0d74-4874-a8f2-d1d46956d697,Asisten Finance <ol><li>Pemahaman Sistem Keuan...,asisten finance <ol><li>pemahaman sistem keuan...
4,b6062e51-4764-474c-a2b7-617affd56959,Marketing Asuransi Mini Mikro <p>WFH dari Sel...,marketing asuransi mini mikro <p>wfh dari sel...
...,...,...,...
677,7a2f6ef1-844d-4c70-99bc-03be7d5aac88,Marketing Creative Intern Responsibilities \r\...,marketing creative intern responsibilities \r\...
678,404ccd37-0c1e-43ab-bdfd-7b9a6fee551c,Graphic Design (Intern) Kami Idea merupakan se...,graphic design (intern) kami idea merupakan se...
679,5a704786-6865-40c9-9086-a2bd85dc5098,Influencer Management Intern - Last semester s...,influencer management intern - last semester s...
680,780902fd-e51a-463c-a012-e7a0941da47e,New Business Development Internship (Jakarta U...,new business development internship (jakarta u...


### **Remove Html Tags**

In [25]:
from lxml import etree

In [26]:
def remove_html_tags(text):
    parser = etree.HTMLParser()
    tree = etree.fromstring(text, parser)
    return etree.tostring(tree, encoding='unicode', method='text')

data['result_remove_html_tags'] = data['result_case_folding_data'].apply(remove_html_tags)
data[['id', 'result_case_folding_data' ,'result_remove_html_tags']]

,id,result_case_folding_data,result_remove_html_tags
0,b794e1a1-ecd6-449d-9ed9-0118a2e02626,website developer <p><strong>kualifikasi :</st...,website developer kualifikasi : wajib memiliki...
1,7c8623ef-5f9b-470d-bc7a-73b1dd706c8b,software architect intern <p>to help architect...,software architect intern to help architect te...
2,bd95c730-b157-4fc5-8791-9dc9ea5e0e32,product marketing intern <p>- design and devel...,product marketing intern - design and develop ...
3,fc01895c-0d74-4874-a8f2-d1d46956d697,asisten finance <ol><li>pemahaman sistem keuan...,asisten finance pemahaman sistem keuangan seko...
4,b6062e51-4764-474c-a2b7-617affd56959,marketing asuransi mini mikro <p>wfh dari sel...,marketing asuransi mini mikro wfh dari seluru...
...,...,...,...
677,7a2f6ef1-844d-4c70-99bc-03be7d5aac88,marketing creative intern responsibilities \r\...,marketing creative intern responsibilities \r\...
678,404ccd37-0c1e-43ab-bdfd-7b9a6fee551c,graphic design (intern) kami idea merupakan se...,graphic design (intern) kami idea merupakan se...
679,5a704786-6865-40c9-9086-a2bd85dc5098,influencer management intern - last semester s...,influencer management intern - last semester s...
680,780902fd-e51a-463c-a012-e7a0941da47e,new business development internship (jakarta u...,new business development internship (jakarta u...


### **Remove Non-Alphanumeric Character**

In [27]:
import re

In [28]:
data['result_remove_non_alphanumeric_character'] = data['result_remove_html_tags'].apply(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x))
data[['id', 'result_remove_html_tags', 'result_remove_non_alphanumeric_character']]

,id,result_remove_html_tags,result_remove_non_alphanumeric_character
0,b794e1a1-ecd6-449d-9ed9-0118a2e02626,website developer kualifikasi : wajib memiliki...,website developer kualifikasi wajib memiliki l...
1,7c8623ef-5f9b-470d-bc7a-73b1dd706c8b,software architect intern to help architect te...,software architect intern to help architect te...
2,bd95c730-b157-4fc5-8791-9dc9ea5e0e32,product marketing intern - design and develop ...,product marketing intern design and develop s...
3,fc01895c-0d74-4874-a8f2-d1d46956d697,asisten finance pemahaman sistem keuangan seko...,asisten finance pemahaman sistem keuangan seko...
4,b6062e51-4764-474c-a2b7-617affd56959,marketing asuransi mini mikro wfh dari seluru...,marketing asuransi mini mikro wfh dari seluru...
...,...,...,...
677,7a2f6ef1-844d-4c70-99bc-03be7d5aac88,marketing creative intern responsibilities \r\...,marketing creative intern responsibilities sup...
678,404ccd37-0c1e-43ab-bdfd-7b9a6fee551c,graphic design (intern) kami idea merupakan se...,graphic design intern kami idea merupakan sebu...
679,5a704786-6865-40c9-9086-a2bd85dc5098,influencer management intern - last semester s...,influencer management intern last semester st...
680,780902fd-e51a-463c-a012-e7a0941da47e,new business development internship (jakarta u...,new business development internship jakarta ut...


### **Tokenization**

In [29]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aszay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
data['result_tokenization'] = data['result_remove_non_alphanumeric_character'].apply(lambda x: word_tokenize(x))
data[['id', 'result_remove_non_alphanumeric_character', 'result_tokenization']]

,id,result_remove_non_alphanumeric_character,result_tokenization
0,b794e1a1-ecd6-449d-9ed9-0118a2e02626,website developer kualifikasi wajib memiliki l...,"[website, developer, kualifikasi, wajib, memil..."
1,7c8623ef-5f9b-470d-bc7a-73b1dd706c8b,software architect intern to help architect te...,"[software, architect, intern, to, help, archit..."
2,bd95c730-b157-4fc5-8791-9dc9ea5e0e32,product marketing intern design and develop s...,"[product, marketing, intern, design, and, deve..."
3,fc01895c-0d74-4874-a8f2-d1d46956d697,asisten finance pemahaman sistem keuangan seko...,"[asisten, finance, pemahaman, sistem, keuangan..."
4,b6062e51-4764-474c-a2b7-617affd56959,marketing asuransi mini mikro wfh dari seluru...,"[marketing, asuransi, mini, mikro, wfh, dari, ..."
...,...,...,...
677,7a2f6ef1-844d-4c70-99bc-03be7d5aac88,marketing creative intern responsibilities sup...,"[marketing, creative, intern, responsibilities..."
678,404ccd37-0c1e-43ab-bdfd-7b9a6fee551c,graphic design intern kami idea merupakan sebu...,"[graphic, design, intern, kami, idea, merupaka..."
679,5a704786-6865-40c9-9086-a2bd85dc5098,influencer management intern last semester st...,"[influencer, management, intern, last, semeste..."
680,780902fd-e51a-463c-a012-e7a0941da47e,new business development internship jakarta ut...,"[new, business, development, internship, jakar..."


### **Remove Stopwords**

In [31]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aszay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
english_stopwords = stopwords.words('english')

stopword_factory = StopWordRemoverFactory()
list_stopwords = ArrayDictionary(stopword_factory.get_stop_words() + english_stopwords)


stopword = StopWordRemover(list_stopwords)

In [33]:
def remove_stopwords(text):
    filtered_words = [stopword.remove(word) for word in text]
    return filtered_words

# Apply remove_stopwords to the 'result_tokenization' column
data['result_remove_stopwords'] = data['result_tokenization'].apply(remove_stopwords)

data['result_remove_stopwords'] = data['result_remove_stopwords'].apply(lambda x: ' '.join(x))

data[['id', 'result_tokenization', 'result_remove_stopwords']]

,id,result_tokenization,result_remove_stopwords
0,b794e1a1-ecd6-449d-9ed9-0118a2e02626,"[website, developer, kualifikasi, wajib, memil...",website developer kualifikasi wajib memiliki l...
1,7c8623ef-5f9b-470d-bc7a-73b1dd706c8b,"[software, architect, intern, to, help, archit...",software architect intern help architect team...
2,bd95c730-b157-4fc5-8791-9dc9ea5e0e32,"[product, marketing, intern, design, and, deve...",product marketing intern design develop share...
3,fc01895c-0d74-4874-a8f2-d1d46956d697,"[asisten, finance, pemahaman, sistem, keuangan...",asisten finance pemahaman sistem keuangan seko...
4,b6062e51-4764-474c-a2b7-617affd56959,"[marketing, asuransi, mini, mikro, wfh, dari, ...",marketing asuransi mini mikro wfh seluruh ind...
...,...,...,...
677,7a2f6ef1-844d-4c70-99bc-03be7d5aac88,"[marketing, creative, intern, responsibilities...",marketing creative intern responsibilities sup...
678,404ccd37-0c1e-43ab-bdfd-7b9a6fee551c,"[graphic, design, intern, kami, idea, merupaka...",graphic design intern idea merupakan sebuah u...
679,5a704786-6865-40c9-9086-a2bd85dc5098,"[influencer, management, intern, last, semeste...",influencer management intern last semester stu...
680,780902fd-e51a-463c-a012-e7a0941da47e,"[new, business, development, internship, jakar...",new business development internship jakarta ut...


### **Stemming**

In [34]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [35]:
stemmer = StemmerFactory().create_stemmer()

data['result_stemming'] = data['result_remove_stopwords'].apply(lambda x: stemmer.stem(x))
data[['id', 'result_remove_stopwords', 'result_stemming']]

,id,result_remove_stopwords,result_stemming
0,b794e1a1-ecd6-449d-9ed9-0118a2e02626,website developer kualifikasi wajib memiliki l...,website developer kualifikasi wajib milik lapt...
1,7c8623ef-5f9b-470d-bc7a-73b1dd706c8b,software architect intern help architect team...,software architect intern help architect team ...
2,bd95c730-b157-4fc5-8791-9dc9ea5e0e32,product marketing intern design develop share...,product marketing intern design develop sharep...
3,fc01895c-0d74-4874-a8f2-d1d46956d697,asisten finance pemahaman sistem keuangan seko...,asisten finance paham sistem uang sekolah ajak...
4,b6062e51-4764-474c-a2b7-617affd56959,marketing asuransi mini mikro wfh seluruh ind...,marketing asuransi mini mikro wfh seluruh indo...
...,...,...,...
677,7a2f6ef1-844d-4c70-99bc-03be7d5aac88,marketing creative intern responsibilities sup...,marketing creative intern responsibilities sup...
678,404ccd37-0c1e-43ab-bdfd-7b9a6fee551c,graphic design intern idea merupakan sebuah u...,graphic design intern idea rupa buah usaha bis...
679,5a704786-6865-40c9-9086-a2bd85dc5098,influencer management intern last semester stu...,influencer management intern last semester stu...
680,780902fd-e51a-463c-a012-e7a0941da47e,new business development internship jakarta ut...,new business development internship jakarta ut...


### **Save Preprocessing**

In [37]:
data.to_csv('../data/cleaned_data.csv', index=False)

## **Data Weighing**

### **Vectorizer**

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
tfidf_vectorizer = TfidfVectorizer()

X = tfidf_vectorizer.fit_transform(data['result_stemming'])

X

<682x6448 sparse matrix of type '<class 'numpy.float64'>'
	with 40541 stored elements in Compressed Sparse Row format>

## **Data Modelling**

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

### **Content Based**

In [41]:
def content_based_recommendation(content_id, n=10):
    # Find the index corresponding to the content_id
    content_index = magang_opportunities.index[magang_opportunities['id'] == content_id].tolist()[0]

    similarity_score = cosine_similarity(X)
    sorted_similar_content = similarity_score[content_index].argsort()[::-1]

    top_n_content = sorted_similar_content[1:n+1]

    recommendation_result = pd.DataFrame(columns=['id', 'name', 'score'])

    print(f"Content magang: {magang_opportunities.loc[magang_opportunities['id'] == content_id, 'name'].iloc[0]}")

    print(f"Top {n} Recommendation result: ")

    for i in top_n_content:
        score = similarity_score[content_index][i]
        if score != 0:  # Check if similarity score is not equal to 0
            recommendation_result = pd.concat([
                recommendation_result,
                pd.DataFrame({
                    'id': [magang_opportunities.iloc[i]['id']],
                    'name': [magang_opportunities.iloc[i]['name']],
                    'score': [score],
                    'mitra': [magang_opportunities.iloc[i]['mitra_name']]
                })
            ], ignore_index=True)

    return recommendation_result

In [42]:
content_based_recommendation('4a70bcd3-f5bf-4496-a892-92893c1f100c', 5)

Content magang: Backend Web Developer - InternPro
Top 5 Recommendation result: 


C:\Users\aszay\AppData\Local\Temp\ipykernel_13132\1228479446.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  recommendation_result = pd.concat([


,id,name,score,mitra
0,b9cdd770-b60b-4de6-a2a1-95189a9c7884,Front End Web Developer - InternPro,0.457408,PT Hanss Business Indonesia
1,e1cf6b2b-a0f8-4544-8820-696ab40d32ed,Quality Assurance Developer - InternPro,0.375144,PT Hanss Business Indonesia
2,a8e3a588-a4a9-4995-bb74-9023b17efb3f,Web Specialist,0.341599,PT SALON KITA INDONESIA
3,54f7f6fa-6a98-4f42-9488-2ba582ea8c96,Front-end Developer Internship,0.287748,LuarKelas
4,aa0c7f9e-0d85-4ce3-8415-19b241bc59b3,Web Development,0.256813,PT FUNVITA INDONESIA INVESTAMA


### **Query Based**

In [43]:
def query_based_recommendation(query, n=10):
    query = query.casefold()  # Make sure the query is in lowercase
    query_vector = tfidf_vectorizer.transform([query])  # Pass the query as a list
    
    similarity_score = cosine_similarity(query_vector, X)
    sorted_similar_content = similarity_score.argsort()[0][::-1]  # Reverse sort indices
    top_n_content = sorted_similar_content[1:n+1]

    recommendation_result = pd.DataFrame(columns=['id', 'name', 'score'])
    print(f"Query: {query}")
    print(f"Top {n} Recommendation result: ")
    for i in top_n_content:
        score = similarity_score[0][i]
        if score != 0:  # Check if similarity score is not equal to 0
            recommendation_result = pd.concat([
                recommendation_result,
                pd.DataFrame({
                    'id': [magang_opportunities.iloc[i]['id']],
                    'name': [magang_opportunities.iloc[i]['name']],
                    'score': [score],
                    'mitra': [magang_opportunities.iloc[i]['mitra_name']]
                })
            ], ignore_index=True)

    return recommendation_result

In [44]:
query_based_recommendation('frontend web developer', 5)

Query: frontend web developer
Top 5 Recommendation result: 


C:\Users\aszay\AppData\Local\Temp\ipykernel_13132\3675382308.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  recommendation_result = pd.concat([


,id,name,score,mitra
0,27b9fcd3-bcee-4cb5-a28c-1080acc66e71,Frontend Developer Intern,0.335844,Koneksi Group
1,4a70bcd3-f5bf-4496-a892-92893c1f100c,Backend Web Developer - InternPro,0.328916,PT Hanss Business Indonesia
2,54f7f6fa-6a98-4f42-9488-2ba582ea8c96,Front-end Developer Internship,0.249744,LuarKelas
3,aa0c7f9e-0d85-4ce3-8415-19b241bc59b3,Web Development,0.228693,PT FUNVITA INDONESIA INVESTAMA
4,e1cf6b2b-a0f8-4544-8820-696ab40d32ed,Quality Assurance Developer - InternPro,0.159811,PT Hanss Business Indonesia
